In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Import necessary Dash components
import dash_leaflet as dl
from dash import dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
import os
import pandas as pd

# Import your CRUD module
from crud import CRUD

username = "aacuser"
password = "aacuser"
db = CRUD(
    host="nv-desktop-services.apporto.com",
    port=34009,
    db_name="AAC",
    collection="animals",
    user=username,
    password=password
)


# Fetch initial data from MongoDB
df = pd.DataFrame.from_records(db.read({}))
if '_id' in df.columns:
    df.drop(columns=['_id'], inplace=True)

# Start Dash app
app = JupyterDash(__name__)

# Encode logo
image_filename = 'GraziosoSalvareLogo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read()).decode()

# Layout
app.layout = html.Div([
    html.Center(html.Img(src=f'data:image/png;base64,{encoded_image}', style={'width': '200px'})),
    html.Center(html.H3("Dashboard by Dylan Miller")),
    html.Center(html.H1('CS-340 Grazioso Salvare Dashboard')),
    html.Hr(),

    # Filtering options
    html.Div([
        html.Label('Rescue Type:'),
        dcc.RadioItems(
            id='filter-type',
            options=[
                {'label': 'Water Rescue', 'value': 'Water Rescue'},
                {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
                {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'},
                {'label': 'Reset', 'value': 'Reset'}
            ],
            value='Reset',
            labelStyle={'display': 'block'}
        )
    ]),
    html.Hr(),

    # Data table
    dash_table.DataTable(
        id='datatable-id',
        columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
        data=df.to_dict('records'),
        page_size=10,
        filter_action='native',
        sort_action='native',
        row_selectable='single',
        selected_rows=[0],
        style_table={'overflowX': 'auto'},
    ),
    html.Br(),
    html.Hr(),

    # Charts and map
    html.Div(className='row', style={'display': 'flex'}, children=[
        html.Div(id='graph-id', className='col s12 m6'),
        html.Div(id='map-id', className='col s12 m6')
    ])
])

# Update data table based on rescue type
@app.callback(
    Output('datatable-id', 'data'),
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    if filter_type == 'Reset':
        query = {}
    elif filter_type == 'Water Rescue':
        query = {"breed": {"$in": ["Labrador Retriever Mix", "Chesapeake Bay Retriever", "Newfoundland"]}}
    elif filter_type == 'Mountain or Wilderness Rescue':
        query = {"breed": {"$in": ["German Shepherd", "Alaskan Malamute", "Old English Sheepdog", "Siberian Husky"]}}
    elif filter_type == 'Disaster or Individual Tracking':
        query = {"breed": {"$in": ["German Shepherd", "Bloodhound", "Rottweiler"]}}
    else:
        query = {}

    filtered = pd.DataFrame.from_records(db.read(query))
    if '_id' in filtered.columns:
        filtered.drop(columns=['_id'], inplace=True)
    return filtered.to_dict('records')

# Update pie chart
@app.callback(
    Output('graph-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data')]
)
def update_graphs(viewData):
    if not viewData:
        return [html.Div("No data to display.")]
    dff = pd.DataFrame(viewData)
    fig = px.pie(dff, names='breed', title='Breed Distribution')
    return [dcc.Graph(figure=fig)]

# Highlight selected columns
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    if selected_columns is None:
        return []  # Return an empty list instead of None
    return [{
        'if': {'column_id': i},
        'background_color': '#D2F3FF'
    } for i in selected_columns]

# Update map
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    if not viewData or not index:
        return [html.Div("No location data available.")]
    dff = pd.DataFrame(viewData)
    row = index[0]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75, -97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[dff.iloc[row, 13], dff.iloc[row, 14]], children=[
                dl.Tooltip(dff.iloc[row, 4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row, 9])
                ])
            ])
        ])
    ]

# Run app
app.run_server(debug=True)


Dash app running on http://127.0.0.1:10260/
